<a href="https://colab.research.google.com/github/sanjay-thiyagarajan/score-predictor/blob/main/CricPredictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget 'https://internalapp.nptel.ac.in/contest/reference_docs/ipl_csv2.zip'
!mkdir '/content/dataset'
!unzip '/content/ipl_csv2.zip' -d '/content/dataset'

In [52]:
import pandas as pd
import numpy as np
import os
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
import warnings
warnings.filterwarnings('ignore')

In [90]:
train_parent = []
DATASET_DIR = '/content/dataset/'
for file in os.listdir(DATASET_DIR):
  filename = os.path.join(DATASET_DIR, file)
  df = pd.read_csv(filename, error_bad_lines=False, header=0)
  train_parent.append(df)
train_parent = pd.concat(train_parent, axis=0, ignore_index=True)

b'Skipping line 7: expected 1 fields, saw 3\nSkipping line 10: expected 1 fields, saw 2\nSkipping line 13: expected 1 fields, saw 3\nSkipping line 26: expected 1 fields, saw 2\nSkipping line 29: expected 1 fields, saw 2\nSkipping line 30: expected 1 fields, saw 2\nSkipping line 33: expected 1 fields, saw 2\nSkipping line 37: expected 1 fields, saw 2\nSkipping line 38: expected 1 fields, saw 2\nSkipping line 39: expected 1 fields, saw 2\nSkipping line 42: expected 1 fields, saw 4\nSkipping line 47: expected 1 fields, saw 2\nSkipping line 74: expected 1 fields, saw 4\nSkipping line 78: expected 1 fields, saw 3\nSkipping line 81: expected 1 fields, saw 2\nSkipping line 84: expected 1 fields, saw 5\nSkipping line 85: expected 1 fields, saw 2\nSkipping line 88: expected 1 fields, saw 2\nSkipping line 89: expected 1 fields, saw 3\nSkipping line 92: expected 1 fields, saw 2\nSkipping line 95: expected 1 fields, saw 2\nSkipping line 924: expected 1 fields, saw 2\nSkipping line 925: expected 1 

In [91]:
power_play_df = train_parent[train_parent['ball']<=5.6]
power_play_df = power_play_df[['match_id','innings', 'striker', 'bowler', 'runs_off_bat', 'extras']]
people = list(np.array(train_parent[['striker']]).flatten()) + list(np.array(train_parent[['bowler']]).flatten())
people = list(set(people))
for p in people:
  power_play_df[p] = [0]*power_play_df.shape[0]

In [92]:
for index, row in power_play_df.iterrows():
  power_play_df[row['striker']][index] = 1
  power_play_df[row['bowler']][index] = 1

In [93]:
d = {'runs_off_bat': 'sum'}
for p in people:
  d[p] = 'mean'
power_play_df = power_play_df.groupby(['match_id', 'innings'], as_index=False).aggregate(d)
power_play_df['runs_off_bat'] /= 2
for p in people:
  power_play_df[p] = np.ceil(np.array(power_play_df[p]))

In [94]:
power_play_df = power_play_df.drop(columns=['match_id', 'innings'])

In [96]:
X_train = np.array(power_play_df.drop(columns=['runs_off_bat']))
Y_train = np.array(power_play_df[['runs_off_bat']])

In [97]:
regressor = LinearRegression(normalize=True)
regressor.fit(X_train, Y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=True)

In [98]:
r2_score(Y_train, regressor.predict(X_train))

0.4222745723658944